# Données COVID-19 Australie: exploration et nettoyage

Test évaluation pour le poste Inria / AP-HP - notebook 1

## 1. Importation des données et des librairies python

In [1]:
#! pip install pandas-dedupe

In [2]:
import pandas as pd
#import pandas_dedupe
import recordlinkage
import seaborn as sns
from sqlalchemy import create_engine
import numpy as np
from sklearn import preprocessing
import time
import datetime
import matplotlib.pyplot as plt

In [3]:
engine = create_engine('sqlite:///data.db', echo=False)
con = engine.connect()
df_patient = pd.read_sql('select * from patient', con=con)#, parse_dates=['date_of_birth']) # dates are parsed into date and time format
df_pcr = pd.read_sql('select * from test', con=con) 
con.close()

OperationalError: (sqlite3.OperationalError) no such table: patient
[SQL: select * from patient]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

### 1.1 Données des patients 

In [ ]:
print("size patients dataframe:",df_patient.shape)
df_patient[:5]

In [ ]:
print('how many patients in total?')
tot_patients = len(df_patient['patient_id'].unique())
print(tot_patients, ' patients') #num. unique IDs


### 1.2 Résultats du PCR, test utilisé pour le diagnostic du Covid19

In [ ]:
print("size pcr dataframe:",df_pcr.shape)
df_pcr.head()

In [ ]:
print('how many PCR in total?')
# list of tested patients
tested_list = df_pcr['patient_id'].unique() #List unique values in the PCR patients ID column
tot_pcr = len(tested_list)
print(tot_pcr, ' PCR') #num. unique IDs

## 2. Nettoyage des données - PCR

In [ ]:
print('variables and types in pcr dataframe:')
print('')
print(df_pcr.dtypes) # variables and types

### 2.1 Doublons

In [ ]:
df_pcr_nodupes = df_pcr.drop_duplicates(subset=None, keep="first", inplace=False)

In [ ]:
duplicateDFRow_pcr = df_pcr_nodupes[df_pcr_nodupes.duplicated()]
print(' \n PCR duplicates \n', duplicateDFRow_pcr)

In [ ]:
duplicateID_pcr = df_pcr_nodupes[df_pcr_nodupes.duplicated(['patient_id'])]
print("we have", len(duplicateID_pcr.index), "duplicated pcr IDs vs.", len(duplicateDFRow_pcr.index), "duplicate rows")

In [ ]:
duplicate_pcr_id = df_pcr_nodupes.loc[df_pcr_nodupes['patient_id'].isin(duplicateID_pcr['patient_id'])]

duplicate_pcr_id.sort_values(by=['patient_id'], inplace=True)

duplicate_pcr_id[:10] #information to be removed since it is contradictory

In [ ]:
df_pcr_clean = df_pcr_nodupes.loc[~df_pcr_nodupes['patient_id'].isin(duplicateID_pcr['patient_id'])]

### 2.2 Valeurs manquantes et standardisation résultats PCR

In [ ]:
print('amount of empty cells in PCR dataframe columns: \n', df_pcr_clean.isnull().sum(axis = 0))

In [ ]:
print(df_pcr_clean['pcr'].unique())

In [ ]:
df_pcr_clean = df_pcr_clean.replace({'pcr': {'P': 'Positive', 'N': 'Negative'}}) # replace P and N with verbose values

In [ ]:
print('before cleaning',df_pcr.shape)
print('after cleaning',df_pcr_clean.shape)
df_pcr_clean.head()

In [ ]:
tested_IDlist_clean = df_pcr_clean['patient_id'].unique()

## 3. Nettoyage des données - patients

- celles vides / valeurs manquantes

- imputation celles vides et traitement des valeurs aberrants 

- typos dans la colonne états (state)

- détection et suppression des doublons


## 3.1 Valeurs manquantes

### 3.1.1 Statistique celles vides

In [ ]:
i = 0 # initialize column count
colnum = []
colname = []
nullpc = []
for col in df_patient:
    i +=1 # update the counter
    nulsum = sum(pd.isnull(df_patient[col])) # sum of null value for the column (empty cells)
    numrows = len(df_patient)
    nuls_pourcent = (sum(pd.isnull(df_patient[col]))/numrows)*100 # % of null value for the column
    r_nuls_pourcent = round(nuls_pourcent, 3) # return only the first 3 digits after the comma of the percentage float value
    #create columnstats database
    colnum.append(i) # first column: column number
    colname.append(col) # column name
    nullpc.append(r_nuls_pourcent) # % null values
    

In [ ]:
df_nul = pd.DataFrame({'num':colnum, 'name':colname, '%null':nullpc})
df_nul = df_nul.sort_values(by='%null', ascending=False)

In [ ]:
plt.figure(figsize=(23,8))

plt.bar(df_nul['name'], df_nul['%null'])
plt.title('NaN % in the dataframe', color="red", fontsize = 14)
plt.ylabel('% empty cell in the column', color="red", fontsize = 14)
plt.xticks(rotation='vertical')
plt.rcParams['figure.constrained_layout.use'] = True
plt.savefig("null.png", format="PNG", dpi = 100)

### 3.1.2 Imputation des valeurs manquantes

In [ ]:
print('variables and types in patients dataframe:')
print('')
print(df_patient.dtypes) # variables and types

#### 3.1.2.1 inputation variables numeriques

#####  - CAP

In [ ]:
df_patient_filled = df_patient #initialize the dataframe to be filled

In [ ]:
# function to fill postcode values
def postcode_filler(df):
    df_patient_filled['postcode'] = df_patient_filled['postcode'].fillna('0000') # fill empty cells
    df_patient_filled['postcode'] = df_patient_filled.apply(lambda cell: '0000' if len(cell['postcode']) > 4 else cell['postcode'], axis=1) # inputation of values that are not a postcode
    df_patient_filled['postcode'] = df_patient_filled['postcode'].astype(int)
    return df

In [ ]:
df_patient_filled = postcode_filler(df_patient_filled) # apply the function 

#####  - numèro rue

In [ ]:
df_patient_filled.street_number.fillna(0) 

print('missing street numbers filled with 0')

#####  - dates de naissance et age

Pour traiter la variable "date de naissance" on pourrait la convertir en variable date comme on ferait avec la commande SQL 

select convert(date, convert(varchar(255), yyyymm) + '01')

mais pour la comparer à l'age est plus interessant recuperer seulement l'année


In [ ]:
def year_from_sqldate(df, datecolumn):
    datecolumn = datecolumn.fillna(19001101.0) # imputation des valeurs mancants avec 1900
    df['dates'] = datecolumn.astype(str)
    df['birth_year'] = [x[:4] for x in df_patient['dates']]
    df['birth_year'] = df['birth_year'].astype(int)
    del df['dates']
    return df['birth_year'] 

In [ ]:
year_from_sqldate(df_patient_filled, df_patient_filled['date_of_birth']) # apply the function to the patients dataframe

In [ ]:
print(min(df_patient_filled['birth_year']), max(df_patient_filled['birth_year'])) 
df_patient_filled['birth_year'] = df_patient_filled.apply(lambda cell: 1900 if cell['birth_year']<1900 else cell['birth_year'], axis=1)
      
print('correct dates earlier than 1900')

In [ ]:
# knowing that COVID-19 came to Australia in 2020, I can calculate the age subctracting the birthyear 

df_patient_filled["estimated_age"] = 2020 - df_patient['birth_year'] 

In [ ]:
# filling missing values with those calculated from birth year
df_patient_filled['age']= df_patient_filled['age'].fillna(df_patient_filled["estimated_age"])
df_patient_filled['age']= df_patient_filled['age'].astype(int)

##### - valeurs aberrants age

In [ ]:
sns.set(style="whitegrid")

ax = sns.boxplot(x=df_patient_filled['age'])

In [ ]:
print('null values:', sum(pd.isnull(df_patient_filled['age'])), "no more empty cells!!!")
print('')
print(len(df_patient_filled.loc[df_patient_filled['age'] < 0]), 'too young to be true')
print(len(df_patient.loc[df_patient_filled['age'] >= 110]), 'too old to be true')
print('max age:',max(df_patient_filled['age']))

In [ ]:
# imputation of average values for excessive values
df_patient_filled['age'] = df_patient_filled.apply(lambda cell: cell['estimated_age'] if cell['age']<0 else cell['age'], axis=1) 
print('ages below 0 are unrealistic, they get replaced by that calculated from birth year')

df_patient_filled['age'] = df_patient_filled.apply(lambda cell: cell['estimated_age'] if cell['age']>110 else cell['age'], axis=1) # imputation of average values for excessive values

print('max age:',max(df_patient_filled['age']))
print('if the patient given age exceeds this value, it is replaced by that calculated from birth year')

#### 3.1.2.2 inputation variables objet

##### - "missing" au lieu des valeurs mancants

In [ ]:
def obj_inputation(df):    
    str_cols = df.select_dtypes(include=['object']).columns

    df.loc[:, str_cols] = df.loc[:, str_cols].fillna('missing')
    return df

df_patient_filled = obj_inputation(df_patient_filled)

##### - la variable état (state)

Beaucoup de typos et valeurs manquants dans la colonne "state". Il ne serait pas possible de faire une statistique basée sur les données brutes. 


On pourrait corriger les typos (ex. nxw = nsw) mais cette methode ne corrigerait pas des nouveaux typos. On cherche une methode plus robuste.

In [ ]:
df_patient_filled['state'].unique()

Pour corriger les erreurs et faire une imputation des valeurs manquants on essaye de recuperer les bonnes valeurs à partir des codes postales.

Ici la liste des code postale associés à chaque état: 

https://en.wikipedia.org/wiki/Postcodes_in_Australia

In [ ]:
df_patient_filled['state_inferred'] = df_patient_filled['postcode'] # initialize feature

In [ ]:
# define lists of postcodes by state:
# New South Wales
NSW1 = np.arange(1000, 2600)
NSW2 = np.arange(2619, 2900)
NSW3 = np.arange(2921, 3000)
NSW = list(np.concatenate((NSW1, NSW2, NSW3), axis=None)) # New South Wales postcodes
#Australian Capital Territory
ACT1 = np.arange(200, 300)
ACT2 = np.arange(2600, 2619)
ACT3 = np.arange(2900, 2921)
ACT = list(np.concatenate((ACT1, ACT2, ACT3), axis=None)) #Australian Capital Territory postcodes
#Victoria
VIC1 = np.arange(3000, 4000)
VIC2 = np.arange(8000, 9000)
VIC = list(np.concatenate((VIC1, VIC2), axis=None)) #Victoria postcodes
#Queensland
QLD1 = np.arange(4000, 5000)
QLD2 = np.arange(9000, 10000)
QLD = list(np.concatenate((QLD1, QLD2), axis=None)) #Queensland postcodes
#South Australia
SA = list(np.arange(5000, 6000)) #South Australia postcodes
#Western Australia
WA = list(np.arange(6000, 7000)) #Western Australia postcodes
#Tasmania
TAS = list(np.arange(7000, 8000)) #Tasmania postcodes
#Northern Territory
NT = list(np.arange(800, 1000)) #Northern Territory postcodes

In [ ]:
states_list = ['nsw', 'act', 'vic', 'qld', 'sa', 'wa', 'tas', 'nt', 'missing']
postcodes_list = [NSW, ACT, VIC, QLD , SA, WA, TAS, NT, [0]]

In [ ]:
# assign state to each postcode in the list 
for i in np.arange(0,len(states_list)):
    df_patient_filled['state_inferred']  = np.where(~df_patient_filled['postcode'].isin(postcodes_list[i]), df_patient_filled['state_inferred'] , states_list[i])
 

Finalement je vais exchanger les typos et les valeurs manquants de la colonne "state" avec ceux de la colonne "inferred state"

In [ ]:
AU_states_list = ['nsw', 'act', 'vic', 'qld', 'sa', 'wa', 'tas', 'nt']

In [ ]:

#df_patient_filled['state']  = df_patient_filled.apply(lambda cell: cell['state_inferred'] if cell['state'].isin(AU_states_list) else cell['state'], axis=1)
df_patient_filled['state_filled']  = np.where(df_patient_filled['state'].isin(AU_states_list), df_patient_filled['state'] , df_patient_filled['state_inferred'])


## 3.2 Doublons

In [ ]:
df_patient_clean = df_patient_filled # initialize the dataframe to get cleaned

### 3.2.1 Exploration des variables clés pour detecter les doublons

#### 3.2.1.1 Lignes identiques

In [ ]:
# this is a function that detects repeated values in the dataframe
duplicateDFRow_patient = df_patient_clean[df_patient_clean.duplicated()]
print('total identical rows: ', len(duplicateDFRow_patient.index))

#### 3.2.1.2 Doublons patient_id

In [ ]:
duplicateID_patient = df_patient_clean[df_patient_clean.duplicated(['patient_id'])]
print("we have", len(duplicateID_patient.index), "duplicated patient IDs vs", len(duplicateDFRow_patient.index), "duplicate rows")

In [ ]:
def detect_duplicates:

In [ ]:
duplicate_patients_id = df_patient_clean.loc[df_patient_clean['patient_id'].isin(duplicateID_patient['patient_id'])]

duplicate_patients_id.sort_values(by=['patient_id'], inplace=True)

duplicate_patients_id[66:70]

In [ ]:
df_patient_clean['tested'] = df_patient_clean['patient_id']

df_patient_clean['tested']  = np.where(df_patient_clean['patient_id'].isin(tested_IDlist_clean), 1 , 0) # assign 1 if patient_ID belongs to tested dataframe, 0 otherwise

In [ ]:
print ('total_IDs labeled as tested in patient dataframe \n before duplicates removal: ', sum(df_patient_clean['tested']),"\n unique tested IDs: ", len(tested_IDlist_clean), "\n duplicate tested IDs:" , sum(df_patient_clean['tested'])-len(tested_IDlist_clean))

In [ ]:
col_list = df_patient_clean.columns.tolist()
#del col_list[1:3]

print(col_list)

#### 3.2.1.2 Doublons nom-prénom-CAP

Du moment que on pourrait avoir des doublons avec numero de telephone manquant, on va créer une autre variable pour identifier au mieux les patients. 

La jointure de nom+prénom donne des doublons mais beaucoup des valeurs identiques

Donc on ajoute le CAP et l'identificatif sera le nom/prenom/CAP des patients.
Pour ca il faut joindre les trois variables

In [ ]:
df_patient_clean['postcode_str']= df_patient_clean['postcode'].astype(str)
df_patient_clean['name_postcode'] = df_patient_clean[['given_name', 'postcode_str']].apply(lambda x: ' '.join(x), axis=1)
df_patient_clean['surname_postcode'] = df_patient_clean[['surname', 'postcode_str']].apply(lambda x: ' '.join(x), axis=1)

In [ ]:
duplicate_pname = df_patient_clean[df_patient_clean.duplicated(['name_postcode'])]
print("we have", len(duplicate_pname.index), "duplicated name+postcode vs", len(duplicateDFRow_patient.index), "duplicate rows")
duplicate_pname_id = df_patient_clean.loc[df_patient_clean['name_postcode'].isin(duplicate_pname['name_postcode'])]

duplicate_pname_id.sort_values(by=['name_postcode'], inplace=True)

In [ ]:
duplicate_psurname = df_patient_clean[df_patient_clean.duplicated(['surname_postcode'])]
print("we have", len(duplicate_psurname.index), "duplicated surname+postcode vs", len(duplicateDFRow_patient.index), "duplicate rows")
duplicate_psurname_id = df_patient_clean.loc[df_patient_clean['surname_postcode'].isin(duplicate_pname['surname_postcode'])]

duplicate_psurname_id.sort_values(by=['surname_postcode'], inplace=True)

In [ ]:
criteria_tested_cap = c1 & c2 & c3
criteria_not_tested_cap = c1 & c2 & c4

In [ ]:
df_duploCAP_tested = inner_join_df[criteria_tested_cap]
print(df_duploCAP_tested.shape)
df_duploCAP_tested.head()

In [ ]:
df_CAP_tested = df_duploCAP_tested.drop_duplicates(subset=['phone_number'], keep='last')
print(df_CAP_tested.shape)
df_CAP_tested[:5]

In [ ]:
# now deduplicate the records for non-tested patients
df_duploCAP_nontested = inner_join_df[criteria_not_tested_cap]
print('before deduplication', df_duploCAP_nontested.shape)
df_CAP_nontested = df_duploCAP_nontested.drop_duplicates(subset=['phone_number'], keep='last')
print('after deduplication', df_CAP_nontested.shape)
df_CAP_nontested[:5]

In [ ]:
c7 = ~ df_CAP_nontested['postcode'].isin(df_CAP_tested['postcode']) 

dfCAP_outer = df_CAP_nontested[c7]
df_no_CAP_dup = df_CAP_tested.append(dfCAP_outer)
print(df_no_CAP_dup.shape)

In [ ]:
duplicate_pname_missingphone = df_no_CAP_dup.loc[df_no_CAP_dup['phone_number'] == 'missing']
print(len(duplicate_pname_missingphone), "duplicate name+surname+CAP with missing phone number")

In [ ]:
c1 = outer_join_df.duplicated(['phone_number']) 
c2 = outer_join_df['postcode'] != "missing"

c3 = outer_join_df['tested'] == 1
c4 = outer_join_df['tested'] == 0
#c5 = df_patient_clean.duplicated(['phone_number']) 
#c6 = df_patient_clean['phone_number'] != "missing"

#### 3.2.1.2 Doublons numéro de téléphone

In [ ]:
duplicate_ph_n = df_patient_clean[df_patient_clean.duplicated(['phone_number']) & (df_patient_clean['phone_number'] != "missing")]
print("we have", len(duplicate_ph_n.index), "duplicated phone numbers vs", len(duplicateDFRow_patient.index), "duplicate rows")
duplicate_phone_id = df_patient_clean.loc[df_patient_clean['phone_number'].isin(duplicate_ph_n['phone_number'])]

duplicate_phone_id.sort_values(by=['phone_number'], inplace=True)

duplicate_phone_id[:6]

In [ ]:
# df_carbon = df_carbon[df_carbon['carbon-footprint_100g'] > 1]
#c3 = df_patient_clean.duplicated(['birth_year'])
df_missingphones = df_patient_clean['phone_number'] == "missing"
#c5 = df_patient_clean['estimated_age'] < 120
duplicate_year_missingphone = df_missingphones[(df_missingphones.duplicated(['birth_year'])) & (df_missingphones['birth_year'] > 1900)]
#duplicate_year_missingphone = duplicate_year_missingphone[df_patient_clean['estimated_age'] < 120]
print("we have", len(duplicate_year_missingphone.index), "duplicated birth years with missing phone numbers vs", len(duplicateDFRow_patient.index), "duplicate rows")
duplicate_year_id = df_patient_clean.loc[df_patient_clean['birth_year'].isin(duplicate_year_missingphone['birth_year'])]

duplicate_year_id.sort_values(by=['birth_year'], inplace=True)

duplicate_year_id[:6]

In [ ]:
criteria_tested_ph = c5 & c6 & c3
criteria_not_tested_ph = c5 & c6 & c4

In [ ]:
df1 = df_patient_clean[criteria_tested_ph]
print(df1.shape)
df1.head()

In [ ]:
df1nodup = df1.drop_duplicates(subset=['phone_number'], keep='last')

In [ ]:
df2 = df_patient_clean[criteria_not_tested_ph]
df2nodup = df2.drop_duplicates(subset=['phone_number'], keep='last')
print(df1nodup.shape, df2nodup.shape)

In [ ]:
#df_no_phone_dup = pd.merge(df1nodup, df2nodup, on='phone_number', how='left')
#print(df_no_phone_dup.shape)

In [ ]:
c7 = ~ df2nodup['phone_number'].isin(df1nodup['phone_number']) 
df2_outer = df2nodup[c7]
df_no_phone_dup = df1nodup.append(df2_outer)
print(df_no_phone_dup.shape)

In [ ]:
df_no_phone_dup.head()

### 3.2.2 Fonction d'élimination des doublons

- on va retenir d'abord les valeurs si le patient-ID est parmi les IDs testés
- en suite on va eliminer les autres doublons
- la procedure est effectuée pour deux variables: numéro de telephone, puis nom+prenom+CAP

![deduplication](deduplication_procedure.png)

In [ ]:
# https://recordlinkage.readthedocs.io/en/latest/notebooks/data_deduplication.html
dfA = df_patient_filled


indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(dfA)

In [ ]:
print (len(dfA), len(candidate_links))
# (1000*1000-1000)/2 = 499500

In [ ]:
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(dfA)

print (len(candidate_links))

In [ ]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()

compare_cl.string('given_name', 'given_name', method='jarowinkler', threshold=0.85, label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('birth_year', 'birth_year', label='birth_year')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA)

In [ ]:
features.head(10)

In [ ]:
features.describe()


In [ ]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)


In [ ]:
# Indexation step
indexer = recordlinkage.Index()
indexer.block(left_on='given_name')
candidate_links = indexer.index(dfA)

# Comparison step
compare_cl = recordlinkage.Compare()

compare_cl.string('given_name', 'given_name', method='jarowinkler', threshold=0.85, label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('birth_year', 'birth_year', label='birth_year')
compare_cl.exact('phone_number', 'phone_number', label='phone_number')
compare_cl.exact('postcode', 'postcode', label='state')
#compare_cl.string('address_1', 'address_1', threshold=0.9, label='address_1')

features = compare_cl.compute(candidate_links, dfA)

In [ ]:
# Classification step
matches = features[features.sum(axis=1) > 3] # at least two matched features
matches = matches.reset_index()
print(len(matches))

In [ ]:
matches.dtypes

In [ ]:
matches.head()